In [ ]:
# train_mask_classifier.py

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

print("TensorFlow Version:", tf.__version__)

# --- Configuration ---
data_dir = "data"  # Directory containing 'with_mask' and 'without_mask' subfolders
img_size = 224
batch_size = 32
num_classes = 2 # 'with_mask' and 'without_mask'

# --- 1. Data Generators ---
# Now, the ImageDataGenerator will split the 'data' directory directly into
# training and validation sets based on the validation_split.
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2, #20% of images for validation
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)

train_generator = datagen.flow_from_directory(
    directory=data_dir, # Point directly to the 'data' folder
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training", # Specify this for the training set
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    directory=data_dir, # Point directly to the 'data' folder again
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation", # Specify this for the validation set
    shuffle=False # Keep validation order consistent
)

# --- CRITICAL: Print Class Indices ---
# This output tells you the exact mapping of class names to integer labels (0, 1)
print("\nClass indices (IMPORTANT - record this for your Streamlit app!):", train_generator.class_indices)
# Expected output based on your folder structure ('with_mask', 'without_mask'):
# {'with_mask': 0, 'without_mask': 1}

# --- 2. Build MobileNetV2 Model ---
base_model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')

# Phase 1: Freeze base model and train the head (new classification layers)
base_model.trainable = False

inputs = Input(shape=(img_size, img_size, 3))
x = base_model(inputs, training=False) # Important: set training=False when base_model is frozen
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x) # Add dropout for regularization
outputs = Dense(num_classes, activation='softmax')(x) # Softmax for 2 classes

model = Model(inputs, outputs)

# --- 3. Compile and Train Phase 1 (Head Only) ---
print("\n--- Training Phase 1: Head Only ---")
model.compile(
    optimizer=Adam(learning_rate=1e-3), # Relatively higher learning rate for the new head
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_phase1 = [
    ModelCheckpoint(
        filepath='mask_classifier_phase1_best.h5', # Save the best model from this phase
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss', # Stop if validation loss doesn't improve
        patience=5, # Number of epochs to wait
        restore_best_weights=True, # Load best weights found
        verbose=1
    )
]

history_phase1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15, # Initial epochs for head training
    callbacks=callbacks_phase1
)

# Load the best model from Phase 1 to ensure we continue fine-tuning from the best weights
model = tf.keras.models.load_model('mask_classifier_phase1_best.h5')

# --- 4. Compile and Train Phase 2 (Full Fine-tuning) ---
print("\n--- Training Phase 2: Full Fine-tuning ---")

# Unfreeze base model for fine-tuning
base_model.trainable = True

# Recompile with a very low learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5), # CRITICAL: Very small learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_phase2 = [
    ModelCheckpoint(
        filepath='mask_detector_model.h5', # This will be your final model saved
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10, # More patience for fine-tuning
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau( # Reduce learning rate if validation loss plateaus
        monitor='val_loss',
        factor=0.2, # Reduce LR by factor of 0.2
        patience=3, # If val_loss doesn't improve for 3 epochs
        min_lr=1e-7, # Don't let LR go below this
        verbose=1
    )
]

history_phase2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50, # More epochs for fine-tuning
    callbacks=callbacks_phase2
)

print("\nMask classifier training complete. Best model saved as 'mask_detector_model.h5'")

TensorFlow Version: 2.19.0
Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.

Class indices (IMPORTANT - record this for your Streamlit app!): {'with_mask': 0, 'without_mask': 1}

--- Training Phase 1: Head Only ---


c:\Users\24ksh\OneDrive\Desktop\mask detec2\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
 58/189 ━━━━━━━━━━━━━━━━━━━━ 1:58 902ms/step - accuracy: 0.6941 - loss: 0.7189

c:\Users\24ksh\OneDrive\Desktop\mask detec2\venv\lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 904ms/step - accuracy: 0.8248 - loss: 0.4219

...

189/189 ━━━━━━━━━━━━━━━━━━━━ 142s 753ms/step - accuracy: 0.9825 - loss: 0.0465 - val_accuracy: 0.9848 - val_loss: 0.0458
Epoch 9/15
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 562ms/step - accuracy: 0.9857 - loss: 0.0434
Epoch 9: val_accuracy improved from 0.98477 to 0.98742, saving model to mask_classifier_phase1_best.h5


189/189 ━━━━━━━━━━━━━━━━━━━━ 193s 704ms/step - accuracy: 0.9857 - loss: 0.0434 - val_accuracy: 0.9874 - val_loss: 0.0342
Epoch 10/15
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 567ms/step - accuracy: 0.9827 - loss: 0.0521
Epoch 10: val_accuracy did not improve from 0.98742
189/189 ━━━━━━━━━━━━━━━━━━━━ 133s 705ms/step - accuracy: 0.9827 - loss: 0.0521 - val_accuracy: 0.9874 - val_loss: 0.0480
Epoch 11/15
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 575ms/step - accuracy: 0.9843 - loss: 0.0452
Epoch 11: val_accuracy did not improve from 0.98742
189/189 ━━━━━━━━━━━━━━━━━━━━ 136s 722ms/step - accuracy: 0.9842 - loss: 0.0452 - val_accuracy: 0.9848 - val_loss: 0.0456
Epoch 12/15
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 567ms/step - accuracy: 0.9792 - loss: 0.0510
Epoch 12: val_accuracy did not improve from 0.98742
189/189 ━━━━━━━━━━━━━━━━━━━━ 134s 708ms/step - accuracy: 0.9792 - loss: 0.0510 - val_accuracy: 0.9841 - val_loss: 0.0511
Epoch 13/15
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 577ms/step - accuracy: 0.9821 - loss: 0.0452
Epoc

189/189 ━━━━━━━━━━━━━━━━━━━━ 145s 769ms/step - accuracy: 0.9821 - loss: 0.0517 - val_accuracy: 0.9881 - val_loss: 0.0474
Epoch 14: early stopping
Restoring model weights from the end of the best epoch: 9.



--- Training Phase 2: Full Fine-tuning ---
Epoch 1/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 585ms/step - accuracy: 0.9841 - loss: 0.0471
Epoch 1: val_accuracy improved from -inf to 0.98808, saving model to mask_detector_model.h5


189/189 ━━━━━━━━━━━━━━━━━━━━ 143s 735ms/step - accuracy: 0.9841 - loss: 0.0471 - val_accuracy: 0.9881 - val_loss: 0.0384 - learning_rate: 1.0000e-05
Epoch 2/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 571ms/step - accuracy: 0.9833 - loss: 0.0471
Epoch 2: val_accuracy did not improve from 0.98808
189/189 ━━━━━━━━━━━━━━━━━━━━ 134s 708ms/step - accuracy: 0.9833 - loss: 0.0471 - val_accuracy: 0.9854 - val_loss: 0.0521 - learning_rate: 1.0000e-05
Epoch 3/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 564ms/step - accuracy: 0.9818 - loss: 0.0522
Epoch 3: val_accuracy did not improve from 0.98808
189/189 ━━━━━━━━━━━━━━━━━━━━ 133s 704ms/step - accuracy: 0.9818 - loss: 0.0522 - val_accuracy: 0.9874 - val_loss: 0.0520 - learning_rate: 1.0000e-05
Epoch 4/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 612ms/step - accuracy: 0.9852 - loss: 0.0430
Epoch 4: val_accuracy did not improve from 0.98808

Epoch 4: ReduceLROnPlateau reducing learning rate to 1.9999999494757505e-06.
189/189 ━━━━━━━━━━━━━━━━━━━━ 144s 762ms/step - accuracy: 0